In [ ]:
import pandas as pd
import numpy as np
import random
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
import folium
from folium.plugins import MarkerCluster

data_temp = pd.read_csv(r'PM2.5國網與中研院台中市夏季晚上.csv', skipinitialspace=True)
GPSdata = pd.read_csv(r'airbox無飄移經緯度.csv', skipinitialspace=True)

cluster_num = 2   # set how many cluster
X = data_temp.T.values[1:]
result = TimeSeriesKMeans(n_clusters = cluster_num, metric="euclidean").fit(X)
labels = result.labels_.tolist()
score = silhouette_score(X, labels, metric="euclidean")

number = 0
color_dict = {}
while number < cluster_num:
    r = lambda: random.randint(0,255)
    tag = number
    color_dict[tag] = '#%02X%02X%02X' % (r(),r(),r())
    number += 1


station = data_temp.columns.tolist()[1:]
total = 0
classification = {}

while True:
    try:
        classification[station[total]] = labels[total]
        total += 1 
    except IndexError:
        break

map_view = folium.Map(location = [22.8, 120.5], zoom_start = 9) 

for sta in station:
    latitude = GPSdata[GPSdata["device_id"] == sta].loc[ : , "latitude"].item()
    longitude = GPSdata[GPSdata["device_id"] == sta].loc[ : , "longitude"].item()

    folium.CircleMarker([latitude, longitude], popup = sta, radius = 6, fill=True, color = color_dict.get(classification.get(sta))).add_to(map_view)

print(score)
map_view